In [1]:
import os, sys, re, argparse, pdb, glob, pprint, json
import TS_pipe as tsp
import pandas as pd
f***REMOVED*** rdkit import Chem
f***REMOVED*** rdkit.Chem import rdmolfiles
f***REMOVED*** openbabel import pybel

json_file = '../results/stilbenes.json'
df = pd.read_json(json_file, orient='index')
df = df.reset_index(names='Molecule')


TS_guess_check = df['TS guess key value'].abs()

TS_count = TS_guess_check.count()
print(TS_count)

121


In [2]:
df

,Molecule,TS mode,key parameter atoms,TS guess key parameter,TS guess key value,pysis input conformers number,pysis output conformers,pysis output conformers number,pysis conformers properties,orca input conformers number,orca output conformers,orca output conformers number,orca conformers properties
0,C14H10Br2,C-C=C-C,"[3, 2, 1, 10]",dihedral C-C=C-C,91.354952,1,[C14H10Br2/8_C14H10Br2_filtered_TS_irc/cregene...,1,{'C14H10Br2/8_C14H10Br2_filtered_TS_irc/cregen...,1,[C14H10Br2/9_C14H10Br2_orca/cregened_conformer...,1,{'C14H10Br2/9_C14H10Br2_orca/cregened_conforme...
1,C14H10BrCl,C-C=C-C,"[3, 2, 1, 10]",dihedral C-C=C-C,91.341397,1,[C14H10BrCl/8_C14H10BrCl_filtered_TS_irc/crege...,1,{'C14H10BrCl/8_C14H10BrCl_filtered_TS_irc/creg...,1,[C14H10BrCl/9_C14H10BrCl_orca/cregened_conform...,1,{'C14H10BrCl/9_C14H10BrCl_orca/cregened_confor...
2,C14H10BrCl_1,C-C=C-C,"[3, 2, 1, 10]",dihedral C-C=C-C,91.326678,1,[C14H10BrCl_1/8_C14H10BrCl_1_filtered_TS_irc/c...,1,{'C14H10BrCl_1/8_C14H10BrCl_1_filtered_TS_irc/...,1,[C14H10BrCl_1/9_C14H10BrCl_1_orca/cregened_con...,1,{'C14H10BrCl_1/9_C14H10BrCl_1_orca/cregened_co...
3,C14H10BrF,C-C=C-C,"[3, 2, 1, 10]",dihedral C-C=C-C,91.334876,1,[C14H10BrF/8_C14H10BrF_filtered_TS_irc/cregene...,1,{'C14H10BrF/8_C14H10BrF_filtered_TS_irc/cregen...,1,[C14H10BrF/9_C14H10BrF_orca/cregened_conformer...,1,{'C14H10BrF/9_C14H10BrF_orca/cregened_conforme...
4,C14H10BrF_1,C-C=C-C,"[3, 2, 1, 10]",dihedral C-C=C-C,91.331167,1,[C14H10BrF_1/8_C14H10BrF_1_filtered_TS_irc/cre...,1,{'C14H10BrF_1/8_C14H10BrF_1_filtered_TS_irc/cr...,1,[C14H10BrF_1/9_C14H10BrF_1_orca/cregened_confo...,1,{'C14H10BrF_1/9_C14H10BrF_1_orca/cregened_conf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,C17H19N_1,C-C=C-C,"[3, 2, 1, 10]",dihedral C-C=C-C,91.022450,2,[C17H19N_1/8_C17H19N_1_filtered_TS_irc/cregene...,2,{'C17H19N_1/8_C17H19N_1_filtered_TS_irc/cregen...,2,[C17H19N_1/9_C17H19N_1_orca/cregened_conformer...,2,{'C17H19N_1/9_C17H19N_1_orca/cregened_conforme...
117,C18H16O4,C-C=C-C,"[3, 2, 1, 13]",dihedral C-C=C-C,91.448620,6,[C18H16O4/8_C18H16O4_filtered_TS_irc/cregened_...,6,{'C18H16O4/8_C18H16O4_filtered_TS_irc/cregened...,6,[C18H16O4/9_C18H16O4_orca/cregened_conformers_...,6,{'C18H16O4/9_C18H16O4_orca/cregened_conformers...
118,C18H19NO2,C-C=C-C,"[3, 2, 1, 12]",dihedral C-C=C-C,91.156337,4,[C18H19NO2/8_C18H19NO2_filtered_TS_irc/cregene...,4,{'C18H19NO2/8_C18H19NO2_filtered_TS_irc/cregen...,4,[C18H19NO2/9_C18H19NO2_orca/cregened_conformer...,4,{'C18H19NO2/9_C18H19NO2_orca/cregened_conforme...
119,C18H19NO2_1,C-C=C-C,"[3, 2, 1, 12]",dihedral C-C=C-C,91.027152,2,[C18H19NO2_1/8_C18H19NO2_1_filtered_TS_irc/cre...,2,{'C18H19NO2_1/8_C18H19NO2_1_filtered_TS_irc/cr...,2,[C18H19NO2_1/9_C18H19NO2_1_orca/cregened_confo...,2,{'C18H19NO2_1/9_C18H19NO2_1_orca/cregened_conf...


In [3]:
failed_orca_count = 0
failed_orca_list = []
conformers_by_orca_count = df['pysis output conformers number'].sum()
conformers_to_orca_count = df['orca input conformers number'].sum()
energies_dict = {}
for i in df.index:
    Z_Es = []
    TS_Es = []
    for j, conf in enumerate(df.loc[i]['orca output conformers']):
        try:
            if not list(df.loc[i]['orca conformers properties'][conf].values()):
                failed_orca_count += 1
                failed_orca_list.append(conf)
            if abs(df.loc[i]['orca conformers properties'][conf]['orca reac1 conformer key value']) < 90.0:
                Z_energy = df.loc[i]['orca conformers properties'][conf]['orca reac1 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac1 E corr']
            else:
                Z_energy = df.loc[i]['orca conformers properties'][conf]['orca reac2 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac2 E corr']
            TS_energy =  df.loc[i]['orca conformers properties'][conf]['orca TS def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca TS E corr']
        except KeyError as e:
            continue
        Z_Es.append(Z_energy)
        TS_Es.append(TS_energy)
    if not ( Z_Es and TS_Es ):
        continue
    energies_dict[f"{df.loc[i]['Molecule']}"] = {}
    energies_dict[f"{df.loc[i]['Molecule']}"]['Z energies'] = Z_Es
    energies_dict[f"{df.loc[i]['Molecule']}"]['TS energies'] = TS_Es
print(f'Conformers generated by ORCA stage: {conformers_to_orca_count}')
print(f'Failed ORCA jobs: {failed_orca_count}')
print(f'Failed ORCA list:', failed_orca_list)
pprint.pprint(energies_dict)
print(len(energies_dict.keys()))

with open('stilbenes_final_energies.json', 'w') as json_file:
    json.dump(energies_dict, json_file, indent=4)



Conformers generated by ORCA stage: 194
Failed ORCA jobs: 1
Failed ORCA list: ['C16H13NO4_1/9_C16H13NO4_1_orca/cregened_conformers_2_reopt']
{'C14H10Br2': {'TS energies': [-5687.870023669123],
               'Z energies': [-5687.9334524681135]},
 'C14H10BrCl': {'TS energies': [-3574.018338451586],
                'Z energies': [-3574.081588432928]},
 'C14H10BrCl_1': {'TS energies': [-3574.0183403556093],
                  'Z energies': [-3574.081585460784]},
 'C14H10BrF': {'TS energies': [-3213.682626411055],
               'Z energies': [-3213.746369020978]},
 'C14H10BrF_1': {'TS energies': [-3213.682626404336],
                 'Z energies': [-3213.746373340858]},
 'C14H10BrI': {'TS energies': [-3411.595292225578],
               'Z energies': [-3411.6586887725207]},
 'C14H10BrI_1': {'TS energies': [-3411.595290935747],
                 'Z energies': [-3411.658686871874]},
 'C14H10BrNO2': {'TS energies': [-3319.009698241624],
                 'Z energies': [-3319.072508545162]},
 'C1

In [4]:
os.getcwd()

'C:\\Users\\***REMOVED***pe672\\Documents\\GitHub\\TS_pipeline\\src'

In [5]:
df2 = pd.read_json('stilbenes_final_energies.json', orient='index')
df2 = df2.explode('TS energies')
df2.index

Index(['C14H10Br2', 'C14H10BrCl', 'C14H10BrCl_1', 'C14H10BrF', 'C14H10BrF_1',
       'C14H10BrI', 'C14H10BrI_1', 'C14H10BrNO2', 'C14H10BrNO2_1', 'C14H10Cl2',
       ...
       'C18H16O4', 'C18H19NO2', 'C18H19NO2', 'C18H19NO2', 'C18H19NO2',
       'C18H19NO2_1', 'C18H19NO2_1', 'C18H22N2', 'C18H22N2', 'C18H22N2'],
      dtype='object', length=193)